In [1]:
import pulp
import pandas as pd
import numpy as np

In [2]:
# Define the problem
prob = pulp.LpProblem("Bus_Allocation", pulp.LpMinimize)

In [3]:
# Define sets
solo_routes = ['ALABANG','BINAN','CARMONA','BALIBAGO','CABUYAO','CALAMBA']  # List of solo routes
hybrid_routes = [('ALABANG','CARMONA'),('BINAN','CARMONA'),('CALAMBA','CABUYAO')]  # List of hybrid routes (each element is a tuple (i, j))

# Import parameters (to be replaced with actual data import code)
cost_small_bus = {}  # Dictionary with costs for small buses on solo routes
cost_large_bus = {}  # Dictionary with costs for large buses on solo routes
cost_small_hybrid_route = {}  # Dictionary with costs for small hybrid routes
cost_large_hybrid_route = {}  # Dictionary with costs for large hybrid routes
capacity_small_bus = 18
capacity_large_bus = 56
buffer_current_small = {}  # Dictionary with buffer capacities for small buses
buffer_current_large = {}  # Dictionary with buffer capacities for large buses
demand = {}  # Dictionary of demands for each route

In [4]:
# Loading cost rates from the spreadsheet
rates_file_path = 'Bus Rate cleaned.xlsx'
xls = pd.ExcelFile(rates_file_path)

# Load the sheet into a DataFrame
cost_df = pd.read_excel(xls, sheet_name=0)

# Extracting costs
for index, row in cost_df.iterrows():
    route = row['ROUTE']
    total_large = row['TOTAL']
    total_small = row['TOTAL.1']
    
    # Check if it's a hybrid route
    if 'VIA' in route:
        route_parts = route.split(' VIA ')
        route_tuple = (route_parts[1], route_parts[0])
        cost_large_hybrid_route[route_tuple] = total_large
        cost_small_hybrid_route[route_tuple] = total_small
    else:
        cost_large_bus[route] = total_large
        cost_small_bus[route] = total_small

In [5]:
# Add bus capacities and buffer size

#Edit this buffer if needed. For now, it's a flat 3 people buffer per bus
buffer_capacity = 3

for r in solo_routes:
    buffer_current_small[r]=buffer_capacity
    buffer_current_large[r]=buffer_capacity
    
for hr in hybrid_routes:
    buffer_current_small[hr]=buffer_capacity
    buffer_current_large[hr]=buffer_capacity

In [6]:
#Demands 

# Load the data
demand_file_path = 'cleaned_bus_shuttle_data_v31.csv'
bus_data = pd.read_csv(demand_file_path)

# Create a dictionary of dictionaries to store the 'ROUTE':'TOTAL' for each shift for each day
shift_data = {}

# Iterate through each unique date
for date in bus_data['DATE'].unique():
    shift_data[date] = {}
    date_data = bus_data[bus_data['DATE'] == date]
    
    # Iterate through each unique shift
    for shift in date_data['SHIFT'].unique():
        shift_data[date][shift] = {}
        shift_data_date_shift = date_data[date_data['SHIFT'] == shift]
        
        # Populate the dictionary for each route and its total
        for _, row in shift_data_date_shift.iterrows():
            route_name = row['ROUTE'].upper()
            shift_data[date][shift][route_name] = row['TOTAL']

# EDIT THIS IF NECESSARY
# Select one of the dictionaries for testing
test_date = '25'  # Select the date
test_shift = 'OUT 4PM'  # Select the shift
test_dict = shift_data[test_date][test_shift]

# Display the selected dictionary for testing
# test_dict

demand = test_dict

In [7]:
demand

{'ALABANG': 4,
 'BINAN': 6,
 'CARMONA': 4,
 'BALIBAGO': 31,
 'TAGAPO': 0,
 'CABUYAO': 4,
 'CALAMBA': 7}

In [8]:
# Preprocessing to exclude NaN values
def preprocess_costs(cost_dict):
    return {k: v for k, v in cost_dict.items() if not np.isnan(v)}

cost_small_bus = preprocess_costs(cost_small_bus)
cost_large_bus = preprocess_costs(cost_large_bus)
cost_small_hybrid_route = preprocess_costs(cost_small_hybrid_route)
cost_large_hybrid_route = preprocess_costs(cost_large_hybrid_route)

# Updated sets based on preprocessed costs
solo_routes = [route for route in solo_routes if route in cost_small_bus or route in cost_large_bus]
hybrid_routes = [route for route in hybrid_routes if route in cost_small_hybrid_route or route in cost_large_hybrid_route]

print("Preprocessed solo routes:", solo_routes)
print("Preprocessed hybrid routes:", hybrid_routes)
print("Preprocessed cost_small_bus:", cost_small_bus)
print("Preprocessed cost_large_bus:", cost_large_bus)
print("Preprocessed cost_small_hybrid_route:", cost_small_hybrid_route)
print("Preprocessed cost_large_hybrid_route:", cost_large_hybrid_route)

Preprocessed solo routes: ['ALABANG', 'BINAN', 'CARMONA', 'BALIBAGO', 'CABUYAO', 'CALAMBA']
Preprocessed hybrid routes: [('ALABANG', 'CARMONA'), ('BINAN', 'CARMONA'), ('CALAMBA', 'CABUYAO')]
Preprocessed cost_small_bus: {'ALABANG': 1423.5, 'BINAN': 586.5, 'BALIBAGO': 1038.0, 'CABUYAO': 586.5, 'CALAMBA': 1300.5}
Preprocessed cost_large_bus: {'ALABANG': 3072, 'BINAN': 2460, 'CARMONA': 1881, 'BALIBAGO': 1881, 'CABUYAO': 1935, 'CALAMBA': 2127}
Preprocessed cost_small_hybrid_route: {('ALABANG', 'CARMONA'): 1518.0, ('BINAN', 'CARMONA'): 630.0, ('CALAMBA', 'CABUYAO'): 630.0}
Preprocessed cost_large_hybrid_route: {('ALABANG', 'CARMONA'): 3270, ('BINAN', 'CARMONA'): 2550, ('CALAMBA', 'CABUYAO'): 2625}


In [9]:
# Define decision variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
h = pulp.LpVariable.dicts("h", hybrid_routes, lowBound=0, upBound=1, cat='Binary')

# Define auxiliary variables for hybrid route capacities
capacity_y_small = pulp.LpVariable.dicts("capacity_y_small", hybrid_routes, lowBound=0, cat='Continuous')
capacity_y_large = pulp.LpVariable.dicts("capacity_y_large", hybrid_routes, lowBound=0, cat='Continuous')

# Objective function
prob += (
    pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] for i in solo_routes if cost_small_bus.get(i) is not None]) +
    pulp.lpSum([cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes if cost_large_bus.get(i) is not None]) +
    pulp.lpSum([cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] for i_j in hybrid_routes if cost_small_hybrid_route.get(i_j) is not None]) +
    pulp.lpSum([cost_large_hybrid_route.get(i_j, 0) * y_large[i_j] for i_j in hybrid_routes if cost_large_hybrid_route.get(i_j) is not None])
)

# Define auxiliary variables to represent hybrid route capacities
for i_j in hybrid_routes:
    if cost_small_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_small[i_j] == y_small[i_j] * (capacity_small_bus - buffer_current_small.get(i_j, 0))
        )
    if cost_large_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_large[i_j] == y_large[i_j] * (capacity_large_bus - buffer_current_large.get(i_j, 0))
        )

# Constraints
# Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        ((capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0) +
        ((capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0) +
        pulp.lpSum([capacity_y_small[i_j] - buffer_current_small.get(i_j, 0) + capacity_y_large[i_j] - buffer_current_small.get(i_j, 0) for i_j in hybrid_routes if i in i_j])
        >= demand.get(i, 0)
    )

# Handling Excess Demand for Route i
for i in solo_routes:
    prob += (
        (x_small[i] * capacity_small_bus) + (x_large[i] * capacity_large_bus) >= 
        demand.get(i, 0) - pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if i in i_j])
    )

# Preventing Overlapping Hybrid Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([h[i_j] for i_j in hybrid_routes if i in i_j]) <= 1
    )

# Cost Efficiency for Hybrid Routes
for i_j in hybrid_routes:
    prob += (
        (cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] + cost_large_hybrid_route.get(i_j, 0) * y_large[i_j]) <= 
        pulp.lpSum([cost_small_bus.get(i, 0) * y_small[i_j] + cost_large_bus.get(i, 0) * y_large[i_j] for i in i_j if cost_small_bus.get(i) is not None and cost_large_bus.get(i) is not None])
    )

# XOR Relationship for Hybrid Route Buses
for i_j in hybrid_routes:
    prob += (
        y_small[i_j] + y_large[i_j] <= 1
    )

# Print the problem data
print("Problem data:")
print(prob)

# Solve the problem
prob.solve()

# Output results
print("\nSolution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Print constraints to verify if they are correct
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint}")


Problem data:
Bus_Allocation:
MINIMIZE
3072*x_large_ALABANG + 1881*x_large_BALIBAGO + 2460*x_large_BINAN + 1935*x_large_CABUYAO + 2127*x_large_CALAMBA + 1881*x_large_CARMONA + 1423.5*x_small_ALABANG + 1038.0*x_small_BALIBAGO + 586.5*x_small_BINAN + 586.5*x_small_CABUYAO + 1300.5*x_small_CALAMBA + 3270*y_large_('ALABANG',_'CARMONA') + 2550*y_large_('BINAN',_'CARMONA') + 2625*y_large_('CALAMBA',_'CABUYAO') + 1518.0*y_small_('ALABANG',_'CARMONA') + 630.0*y_small_('BINAN',_'CARMONA') + 630.0*y_small_('CALAMBA',_'CABUYAO') + 0.0
SUBJECT TO
_C1: capacity_y_small_('ALABANG',_'CARMONA')
 - 15 y_small_('ALABANG',_'CARMONA') = 0

_C2: capacity_y_large_('ALABANG',_'CARMONA')
 - 53 y_large_('ALABANG',_'CARMONA') = 0

_C3: capacity_y_small_('BINAN',_'CARMONA') - 15 y_small_('BINAN',_'CARMONA')
 = 0

_C4: capacity_y_large_('BINAN',_'CARMONA') - 53 y_large_('BINAN',_'CARMONA')
 = 0

_C5: capacity_y_small_('CALAMBA',_'CABUYAO')
 - 15 y_small_('CALAMBA',_'CABUYAO') = 0

_C6: capacity_y_large_('CALAMBA'

# Define decision variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
h = pulp.LpVariable.dicts("h", hybrid_routes, lowBound=0, upBound=1, cat='Binary')

# Define auxiliary variables for hybrid route capacities
capacity_y_small = pulp.LpVariable.dicts("capacity_y_small", hybrid_routes, lowBound=0, cat='Continuous')
capacity_y_large = pulp.LpVariable.dicts("capacity_y_large", hybrid_routes, lowBound=0, cat='Continuous')

# Objective function
prob += (
    pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] for i in solo_routes if cost_small_bus.get(i) is not None]) +
    pulp.lpSum([cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes if cost_large_bus.get(i) is not None]) +
    pulp.lpSum([cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] for i_j in hybrid_routes if cost_small_hybrid_route.get(i_j) is not None]) +
    pulp.lpSum([cost_large_hybrid_route.get(i_j, 0) * y_large[i_j] for i_j in hybrid_routes if cost_large_hybrid_route.get(i_j) is not None])
)

# Define auxiliary variables to represent hybrid route capacities
for i_j in hybrid_routes:
    if cost_small_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_small[i_j] == y_small[i_j] * (capacity_small_bus - buffer_current_small.get(i_j, 0))
        )
    if cost_large_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_large[i_j] == y_large[i_j] * (capacity_large_bus - buffer_current_large.get(i_j, 0))
        )

# Constraints
# Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if cost_small_bus.get(i) is not None else 0) +
        (capacity_large_bus * x_large[i] if cost_large_bus.get(i) is not None else 0) +
        pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if i in i_j])
        >= demand.get(i, 0)
    )

# Handling Excess Demand for Route i
for i in solo_routes:
    prob += (
        (x_small[i] * capacity_small_bus) + (x_large[i] * capacity_large_bus) >= 
        demand.get(i, 0) - pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if i in i_j])
    )

# Handling Excess Demand for Route j
for j in solo_routes:
    prob += (
        (x_small[j] * capacity_small_bus) + (x_large[j] * capacity_large_bus) >= 
        demand.get(j, 0) - pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if j in i_j])
    )

# Preventing Overlapping Hybrid Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([h[i_j] for i_j in hybrid_routes if i in i_j]) <= 1
    )

# Cost Efficiency for Hybrid Routes
for i_j in hybrid_routes:
    prob += (
        (cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] + cost_large_hybrid_route.get(i_j, 0) * y_large[i_j]) <= 
        pulp.lpSum([cost_small_bus.get(i, 0) * y_small[i_j] + cost_large_bus.get(i, 0) * y_large[i_j] for i in i_j if cost_small_bus.get(i) is not None and cost_large_bus.get(i) is not None])
    )

# Capacity Non-Negativity
# Ensured by the definition of variables with lowBound=0

# Binary Variables for Hybrid Routes
# Ensured by the definition of variables as Binary

# XOR Relationship for Hybrid Route Buses
for i_j in hybrid_routes:
    prob += (
        y_small[i_j] + y_large[i_j] <= 1
    )

# Print the problem data
print("Problem data:")
print(prob)

# Solve the problem
prob.solve()

# Output results
print("\nSolution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Print constraints to verify if they are correct
#for name, constraint in prob.constraints.items():
#   print(f"{name}: {constraint}")

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Define decision variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, upBound=1, cat='Binary')
h = pulp.LpVariable.dicts("h", hybrid_routes, lowBound=0, upBound=1, cat='Binary')

# Define auxiliary variables for hybrid route capacities
capacity_y_small = pulp.LpVariable.dicts("capacity_y_small", hybrid_routes, lowBound=0, cat='Continuous')
capacity_y_large = pulp.LpVariable.dicts("capacity_y_large", hybrid_routes, lowBound=0, cat='Continuous')

# Objective function
prob += (
    pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] for i in solo_routes if cost_small_bus.get(i) is not None]) +
    pulp.lpSum([cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes if cost_large_bus.get(i) is not None]) +
    pulp.lpSum([cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] for i_j in hybrid_routes if cost_small_hybrid_route.get(i_j) is not None]) +
    pulp.lpSum([cost_large_hybrid_route.get(i_j, 0) * y_large[i_j] for i_j in hybrid_routes if cost_large_hybrid_route.get(i_j) is not None])
)

# Constraints
# Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if cost_small_bus.get(i) is not None else 0) +
        (capacity_large_bus * x_large[i] if cost_large_bus.get(i) is not None else 0) +
        pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if i in i_j])
        >= demand.get(i, 0)
    )

# Define auxiliary variables to represent hybrid route capacities
for i_j in hybrid_routes:
    if cost_small_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_small[i_j] == y_small[i_j] * (capacity_small_bus - buffer_current_small.get(i_j, 0))
        )
    if cost_large_hybrid_route.get(i_j) is not None:
        prob += (
            capacity_y_large[i_j] == y_large[i_j] * (capacity_large_bus - buffer_current_large.get(i_j, 0))
        )

# Handling Excess Demand for Route i
for i in solo_routes:
    prob += (
        (x_small[i] + x_large[i]) * max(capacity_small_bus, capacity_large_bus) >= 
        demand.get(i, 0) - pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if i in i_j])
    )

# Handling Excess Demand for Route j
for j in solo_routes:
    prob += (
        (x_small[j] + x_large[j]) * max(capacity_small_bus, capacity_large_bus) >= 
        demand.get(j, 0) - pulp.lpSum([capacity_y_small[i_j] + capacity_y_large[i_j] for i_j in hybrid_routes if j in i_j])
    )

# Preventing Overlapping Hybrid Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([h[i_j] for i_j in hybrid_routes if i in i_j]) <= 1
    )

# Cost Efficiency for Hybrid Routes
for i_j in hybrid_routes:
    prob += (
        (cost_small_hybrid_route.get(i_j, 0) * y_small[i_j] + cost_large_hybrid_route.get(i_j, 0) * y_large[i_j]) <= 
        pulp.lpSum([cost_small_bus.get(i, 0) * y_small[i_j] + cost_large_bus.get(i, 0) * y_large[i_j] for i in i_j if cost_small_bus.get(i) is not None and cost_large_bus.get(i) is not None])
    )

# Capacity Non-Negativity
# Ensured by the definition of variables with lowBound=0

# Binary Variables for Hybrid Routes
# Ensured by the definition of variables as Binary

# XOR Relationship for Hybrid Route Buses
for i_j in hybrid_routes:
    prob += (
        y_small[i_j] + y_large[i_j] <= 1
    )

# Zero Demand Constraint
for i in solo_routes:
    if demand.get(i, 0) == 0:
        prob += (x_small[i] == 0)
        prob += (x_large[i] == 0)
for i_j in hybrid_routes:
    if demand.get(i_j, 0) == 0:
        prob += (y_small[i_j] == 0)
        prob += (y_large[i_j] == 0)

# Print the problem data
print("Problem data:")
print(prob)

# Solve the problem
prob.solve()

# Output results
print("\nSolution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))